<a href="https://colab.research.google.com/github/Jordy-20035/Kaggle_MultiClassification/blob/main/Mushroom_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q albumentations imbalanced-learn

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau,
    ModelCheckpoint, CSVLogger, LearningRateScheduler
)
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path
import albumentations as A

In [2]:
# Configuration
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 100
DATA_DIR = Path("/content/drive/MyDrive/dataset/")
MODEL_DIR = Path("/content/drive/MyDrive/Models/MushroomClassification/")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Data Preparation
def load_dataframe(csv_path):
    df = pd.read_csv(csv_path)
    df["Image"] = df["Image"].astype(str).str.zfill(5) + ".jpg"
    return df

In [4]:
train_df = load_dataframe("/train.csv")
test_df = load_dataframe("/test.csv")

# Handle class imbalance in train_df
ros = RandomOverSampler(sampling_strategy='auto')
train_df, _ = ros.fit_resample(train_df, train_df['Mushroom'])

# Train/Val split
train_df, val_df = train_test_split(
    train_df,
    test_size=0.15,
    stratify=train_df['Mushroom'],
    random_state=42
)

In [5]:
# Class weights calculation
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_df['Mushroom']),
    y=train_df['Mushroom']
)
class_weights = dict(enumerate(class_weights))

# Albumentations augmentations
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    #A.Cutout(num_holes=8, max_h_size=24, max_w_size=24, fill_value=0, p=0.5),
])


In [6]:
def augment_image(image):
    image = image.numpy()  # Convert TensorFlow tensor to NumPy array
    augmented_image = aug(image=image)['image']
    return augmented_image

In [7]:

# Data Pipeline
def load_image(image_path, label=None):
    data_dir_str = str(DATA_DIR)  # Convert Path to string
    full_path = tf.strings.join([data_dir_str, image_path], separator='/')

    image = tf.io.read_file(full_path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.py_function(augment_image, [image], Tout=tf.float32)
    image.set_shape(IMAGE_SIZE + (3,))
    image = image / 255.0
    if label is not None:
        return image, label
    else:
        return image

In [8]:
def create_dataset(df, labels=None, shuffle=False):
    if labels is not None:
        ds = tf.data.Dataset.from_tensor_slices(
            (df['Image'].values, labels)
        )
    else:
        ds = tf.data.Dataset.from_tensor_slices(
            df['Image'].values
        )

    if shuffle:
        if labels is not None:
            ds = ds.shuffle(buffer_size=len(df)*2, reshuffle_each_iteration=True)
        else:
            ds = ds.shuffle(buffer_size=len(df), reshuffle_each_iteration=True)

    if labels is not None:
        ds = ds.map(lambda x, y: load_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

    if labels is not None:
        ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    else:
        ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return ds


In [9]:
print(train_df.columns)

Index(['Image', 'Mushroom'], dtype='object')


In [10]:
train_ds = create_dataset(train_df, labels=train_df['Mushroom'], shuffle=True)
val_ds = create_dataset(val_df, labels=val_df['Mushroom'])
test_ds = create_dataset(test_df)

In [11]:
# Custom F1 Metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=1)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [12]:
print(DATA_DIR)

/content/drive/MyDrive/dataset


In [13]:
!ls {DATA_DIR}

00001.jpg  00332.jpg  00663.jpg  00994.jpg  01325.jpg  01656.jpg  01987.jpg  02318.jpg	02649.jpg
00002.jpg  00333.jpg  00664.jpg  00995.jpg  01326.jpg  01657.jpg  01988.jpg  02319.jpg	02650.jpg
00003.jpg  00334.jpg  00665.jpg  00996.jpg  01327.jpg  01658.jpg  01989.jpg  02320.jpg	02651.jpg
00004.jpg  00335.jpg  00666.jpg  00997.jpg  01328.jpg  01659.jpg  01990.jpg  02321.jpg	02652.jpg
00005.jpg  00336.jpg  00667.jpg  00998.jpg  01329.jpg  01660.jpg  01991.jpg  02322.jpg	02653.jpg
00006.jpg  00337.jpg  00668.jpg  00999.jpg  01330.jpg  01661.jpg  01992.jpg  02323.jpg	02654.jpg
00007.jpg  00338.jpg  00669.jpg  01000.jpg  01331.jpg  01662.jpg  01993.jpg  02324.jpg	02655.jpg
00008.jpg  00339.jpg  00670.jpg  01001.jpg  01332.jpg  01663.jpg  01994.jpg  02325.jpg	02656.jpg
00009.jpg  00340.jpg  00671.jpg  01002.jpg  01333.jpg  01664.jpg  01995.jpg  02326.jpg	02657.jpg
00010.jpg  00341.jpg  00672.jpg  01003.jpg  01334.jpg  01665.jpg  01996.jpg  02327.jpg	02658.jpg
00011.jpg  00342.jpg  00673.jp

In [14]:
# Model Architecture
def create_model():
    base_model = EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=IMAGE_SIZE + (3,),
        pooling='avg'
    )
    base_model.trainable = False

    inputs = layers.Input(shape=IMAGE_SIZE + (3,))
    x = base_model(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='swish',
                    kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.6)(x)
    x = layers.Dense(256, activation='swish',
                    kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=Adam(learning_rate=3e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy', F1Score()]
    )
    return model

In [15]:
# Callbacks
callbacks = [
    ModelCheckpoint(
        MODEL_DIR/'best_model.h5',
        save_best_only=True,
        monitor='val_f1_score',
        mode='max',
        save_weights_only=False
    ),
    ReduceLROnPlateau(
        monitor='val_f1_score',
        mode='max',
        factor=0.5,
        patience=5,
        min_lr=1e-6
    ),
    EarlyStopping(
        monitor='val_f1_score',
        patience=15,
        mode='max',
        restore_best_weights=True
    ),
    CSVLogger(MODEL_DIR/'training_log.csv'),
    LearningRateScheduler(lambda epoch: 3e-4 * 0.95 ** epoch)
]

In [16]:
# Training
model = create_model()
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 979ms/step - accuracy: 0.0970 - f1_score: 0.8958 - loss: 2.7935

64/64 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.0970 - f1_score: 0.8959 - loss: 2.7930 - val_accuracy: 0.1000 - val_f1_score: 0.9474 - val_loss: 2.4139 - learning_rate: 3.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 74s 374ms/step - accuracy: 0.1260 - f1_score: 0.9007 - loss: 2.6747 - val_accuracy: 0.1000 - val_f1_score: 0.9441 - val_loss: 2.4136 - learning_rate: 2.8500e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 323ms/step - accuracy: 0.1023 - f1_score: 0.9049 - loss: 2.6245 - val_accuracy: 0.1167 - val_f1_score: 0.9443 - val_loss: 2.4119 - learning_rate: 2.7075e-04
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 23s 355ms/step - accuracy: 0.1028 - f1_score: 0.8932 - loss: 2.5877 - val_accuracy: 0.0889 - val_f1_score: 0.9265 - val_loss: 2.4119 - learning_rate: 2.5721e-04
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 21s 321ms/step - accuracy: 0.0882 - f1_score: 0.9183 - loss: 2.5828 - val_accuracy: 0.0694 - val_f1_score: 0.9427 - val_loss: 2.4123 - learning_rate: 2.4435e-04
Epoch 6/100
64

64/64 ━━━━━━━━━━━━━━━━━━━━ 26s 412ms/step - accuracy: 0.1248 - f1_score: 0.9076 - loss: 2.4994 - val_accuracy: 0.1083 - val_f1_score: 0.9501 - val_loss: 2.4058 - learning_rate: 1.6211e-04
Epoch 14/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 36s 329ms/step - accuracy: 0.0886 - f1_score: 0.9099 - loss: 2.5070 - val_accuracy: 0.1056 - val_f1_score: 0.9427 - val_loss: 2.4164 - learning_rate: 1.5400e-04
Epoch 15/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 22s 342ms/step - accuracy: 0.1065 - f1_score: 0.8947 - loss: 2.4940 - val_accuracy: 0.1111 - val_f1_score: 0.9381 - val_loss: 2.4034 - learning_rate: 1.4630e-04
Epoch 16/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 23s 361ms/step - accuracy: 0.0993 - f1_score: 0.8916 - loss: 2.5048 - val_accuracy: 0.1028 - val_f1_score: 0.6895 - val_loss: 2.4107 - learning_rate: 1.3899e-04
Epoch 17/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 41s 361ms/step - accuracy: 0.0961 - f1_score: 0.8999 - loss: 2.4776 - val_accuracy: 0.0972 - val_f1_score: 0.9474 - val_loss: 2.4109 - learning_rate: 1.3204e-04
Epoch 18

In [17]:
# Fine-tuning
model.load_weights(MODEL_DIR/'best_model.h5')
model.trainable = True
for layer in model.layers[1].layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', F1Score()]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=int(EPOCHS*0.3),
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 92s 804ms/step - accuracy: 0.1126 - f1_score: 0.9183 - loss: 2.5705 - val_accuracy: 0.1139 - val_f1_score: 0.9231 - val_loss: 2.4074 - learning_rate: 3.0000e-04
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 34s 357ms/step - accuracy: 0.0838 - f1_score: 0.9134 - loss: 2.5846 - val_accuracy: 0.0972 - val_f1_score: 0.9443 - val_loss: 2.4237 - learning_rate: 2.8500e-04
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 23s 358ms/step - accuracy: 0.1076 - f1_score: 0.8731 - loss: 2.5341 - val_accuracy: 0.1056 - val_f1_score: 0.9379 - val_loss: 2.4143 - learning_rate: 2.7075e-04
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 24s 373ms/step - accuracy: 0.1043 - f1_score: 0.8959 - loss: 2.5486 - val_accuracy: 0.1083 - val_f1_score: 0.6641 - val_loss: 2.4096 - learning_rate: 2.5721e-04
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 25s 385ms/step - accuracy: 0.0952 - f1_score: 0.8995 - loss: 2.5292 - val_accuracy: 0.1028 - val_f1_score: 0.9474 - val_loss: 2.4079 - learning_rate: 2.4435e-04
Epoch

In [18]:
# Test-Time Augmentation
def predict_with_tta(dataset, model, n_tta=5):
    all_preds = []
    for _ in range(n_tta):
        preds = model.predict(dataset, verbose=0)
        all_preds.append(preds)
    avg_preds = np.mean(all_preds, axis=0)
    final_preds = np.argmax(avg_preds, axis=1)
    return final_preds

In [19]:
# Generate Predictions
test_preds = predict_with_tta(test_ds, model, n_tta=7)

# Create Submission
submission = pd.DataFrame({
    'Id': test_df['Image'].str.replace('.jpg', ''),
    'Predicted': test_preds
})

# Submission

In [20]:
# Ensure exactly 598 entries
submission = submission.head(598)
submission.to_csv(MODEL_DIR/'submission.csv', index=False)
print("Submission saved successfully!")

Submission saved successfully!


In [23]:
print(submission)

        Id  Predicted
0    02372          5
1    02373          5
2    02374          5
3    02375          5
4    02376          8
..     ...        ...
593  02967          5
594  02968          5
595  02969          8
596  02970          5
597  02971          5

[598 rows x 2 columns]


# F1 score

In [22]:
# Print F1 score for training and validation sets
print("Training F1 Score:", history.history['f1_score'][-1])
print("Validation F1 Score:", history.history['val_f1_score'][-1])

Training F1 Score: 0.9086006283760071
Validation F1 Score: 0.9425625205039978
